In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=88014ecdaa55fbbf2f8a22c12d95b7a3609c117ba9b850bc421f405ae7e94bbb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
from torch.optim import AdamW
from tqdm import tqdm
from datasets import Dataset

In [88]:
df = pd.read_csv("/kaggle/input/ch-play/clean_data_v4.csv")

In [89]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,content,score,thumbsUpCount,Application,spell_corrected_content,spam
0,0,0,0,không làm được,1,1,Messenger,không làm được,clean
1,1,1,1,đăng xuất ra vô lại bị lỗi nhìu,1,1,Messenger,đăng xuất ra vô lại bị lỗi nhiều,clean
2,3,3,3,1 sao vì chặn mà vẫn gỡ được,1,0,Messenger,1 sào vì chăn mà vẫn gỡ được,clean
3,4,4,4,sản năm sao luân,5,1,Messenger,sạn năm sao luận,clean
4,6,6,6,mất ft hoài,1,1,Messenger,mắt ft hoài,clean
...,...,...,...,...,...,...,...,...,...
47842,91993,91993,91993,ứng dụng tốt theo dõi và làm quen được nhiều b...,4,0,Instagram,ứng dụng tốt theo dõi và làm quen được nhiều b...,clean
47843,91994,91994,91994,cập nhật không được ạ,4,0,Instagram,cập nhật không được ạ,clean
47844,91996,91996,91996,chụp đẹp nhưng chưa có tính năng giây nha mong...,4,0,Instagram,chụp đẹp nhưng chưa có tính năng giấy nhà mong...,clean
47845,91997,91997,91997,điện video bị lỗi,1,0,Instagram,điện video bị lỗi,clean


In [95]:
dataset = Dataset.from_pandas(df)

trainval_test = dataset.train_test_split(test_size=0.2, seed=0)

trainval_dataset = trainval_test["train"]
test_dataset = trainval_test["test"]

train_val = trainval_dataset.train_test_split(test_size=0.125, seed=0)
train_dataset = train_val["train"]
val_dataset = train_val["test"]

In [96]:
train_dataset = train_dataset.to_pandas()
score2label = {1:0,2:0,3:1, 4:2,5:2}
train_dataset["class"] = train_dataset["score"].apply(lambda x: score2label[x])

In [97]:
train_dataset["class"].value_counts()

class
0    21398
2     9849
1     2245
Name: count, dtype: int64

In [99]:
lst = [train_dataset]
max_size = 10000#train_dataset['class'].value_counts().max()

for class_index, group in train_dataset.groupby('class'):
    lst.append(group.sample(max_size-len(group) if max_size > len(group) else len(group), replace=True))
train_dataset = pd.concat(lst)
train_dataset = Dataset.from_pandas(train_dataset)

In [100]:
train_dataset.to_pandas()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,content,score,thumbsUpCount,Application,spell_corrected_content,spam,class,__index_level_0__
0,43849,43849,43849,không đăng nhập nick được,1,0,Facebook,không đăng nhập nick được,clean,0,0
1,25149,25149,25149,ko vô đc mà màn hình đen thui,5,0,Messenger,không vô được mà màn hình đen thui,clean,2,1
2,18410,18410,18410,khá tiện dụng,5,0,Messenger,khá tiện dụng,clean,2,2
3,21702,21702,21702,sao ko gửi đc ảnh vậy,2,0,Messenger,sao không gửi được ảnh vậy,clean,0,3
4,25068,25068,25068,cập nhật xong gọi video mà không có camera,1,2,Messenger,cập nhật xong gọi video mà không có camera,clean,0,4
...,...,...,...,...,...,...,...,...,...,...,...
62791,1264,1264,1264,ứng dụng này rất tốt và tiện lợi,5,0,Messenger,ứng dụng này rất tốt và tiện lợi,clean,2,265
62792,24869,24869,24869,gần mực thì đen gần đèn thì sáng,5,2,Messenger,gần mực thì đen gần đèn thì sáng,clean,2,748
62793,37569,37569,37569,đc nhắm nhun,5,0,Facebook,được nhắm nhún,clean,2,1248
62794,31671,31671,31671,ok ngon,5,0,Facebook,ô ngon,clean,2,3241


In [101]:
class review_dataset(Dataset):
    
    def __init__(self, df):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
        self.score2label = {1:0,2:0,3:1, 4:2,5:2}
        
    def __len__(self,):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]["spell_corrected_content"]
        score = self.df.iloc[idx]["score"]
        encode = self.tokenizer(text, max_length=512)
        return {"input_ids":encode.input_ids, "attention_mask":encode.attention_mask, "labels":self.score2label[score]}
    
    def collate_fn(batch):
        def get_batch(batch):
            new_batch = {"input_ids":[], "attention_mask":[], "labels":[]}
            for i in batch:
                for k in i:
                    new_batch[k].append(i[k])
            return new_batch
        batch = get_batch(batch)
        max_length = max([len(i) for i in batch["input_ids"]])
        batch["input_ids"] = torch.tensor([i+[0]*(max_length-len(i)) for i in batch["input_ids"]])
        batch["attention_mask"] = torch.tensor([i+[0]*(max_length-len(i)) for i in batch["attention_mask"]])
        arr = torch.zeros(len(batch["labels"]), 3)
        labels = torch.tensor(batch["labels"])
        arr[torch.arange(labels.size(0)), labels] = 1
        arr = 2*(arr-0.5)
        batch["labels"] = arr
        return batch

In [102]:
train_dataset = review_dataset(train_dataset.to_pandas())
test_dataset = review_dataset(test_dataset.to_pandas())
dev_dataset = review_dataset(val_dataset.to_pandas())

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=review_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=review_dataset.collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=True, collate_fn=review_dataset.collate_fn)

In [103]:
next(iter(train_loader))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[    0,   210,  3438, 52615,   723,  3773,  1517,     2,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [    0,    10,   224,  5447,  2952,  8618,  2087,    17,    11,     2,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [    0,  6173,  6032,  5447,  2952,    11,    17,   409,  1624,    89,
           3438,  4525,   236,     2,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [    0,    17,   167,    13,   957,     2,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,  

In [104]:
encoder = AutoModel.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

In [105]:
class svm_sbert(torch.nn.Module):
    
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.svm = torch.nn.Linear(768,3)
    
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def forward(self, input_ids, attention_mask, labels):
        features = self.mean_pooling(self.encoder(input_ids, attention_mask), attention_mask)
        outputs = self.svm(features)
        return torch.sum(torch.clamp(1 - outputs*labels, min=0))/outputs.size(0)
    
    def predict(self, input_ids, attention_mask, labels):
        features = self.encoder(input_ids, attention_mask)[0].mean(axis=1)
        outputs = self.svm(features)
        preds = torch.argmax(outputs, axis=1)
        return preds, torch.argmax(labels, axis=1)

In [106]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [107]:
model = svm_sbert(encoder)

In [108]:
opt1 = AdamW(model.encoder.parameters(), lr=2e-6, weight_decay=1e-2)
opt2 = AdamW(model.svm.parameters(), lr=5e-5, weight_decay=0.5)

In [109]:
metric = load_metric("f1")

In [110]:
def evaluate(loader, model, metric):
    model.eval()
    preds, labels = [], []
    iters = tqdm(loader)
    for batch in iters:
        for k in batch:
            batch[k] = batch[k].to(device)
        pred, label = model.predict(**batch)
        preds.append(pred)
        labels.append(label)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    return metric.compute(predictions=preds, references=labels, average="macro"), (preds==labels).float().mean().item()

In [111]:
model.to(device)
print()

In [112]:
best_dev = 0
dev_acc = 0
test_acc = 0
for epoch in range(10):
    iters = tqdm(train_loader)
    avg_loss = 0
    model.train()
    for idx, batch in enumerate(iters):
        for k in batch:
            batch[k] = batch[k].to(device)
        loss = model(**batch)
        loss.backward()
        opt1.step()
        opt2.step()
        opt1.zero_grad()
        opt2.zero_grad()
        avg_loss = (idx*avg_loss + loss)/(idx+1)
        iters.set_postfix({"loss":avg_loss})
        
    dev_f1, acc = evaluate(dev_loader, model, metric)
    dev_f1 = dev_f1["f1"]
    if dev_f1 > best_dev:
        dev_acc = acc
        best_dev = dev_f1
        test_f1, acc = evaluate(test_loader, model, metric)#["f1"]
        test_f1 = test_f1["f1"]
        test_acc = acc
        print("dev f1: " + str(best_dev) + " test f1: " + str(test_f1))
        print("dev acc: " + str(dev_acc) + " test acc: " + str(test_acc))
    

100%|██████████| 300/300 [00:07<00:00, 39.37it/s]


dev f1: 0.5515555603192069 test f1: 0.5514337416377197
dev acc: 0.8292580842971802 test acc: 0.8233019709587097


100%|██████████| 300/300 [00:07<00:00, 39.32it/s]


dev f1: 0.562459749684502 test f1: 0.5633562632459567
dev acc: 0.8334378004074097 test acc: 0.8290491104125977


100%|██████████| 300/300 [00:07<00:00, 39.21it/s]


...

100%|██████████| 300/300 [00:07<00:00, 39.22it/s]


dev f1: 0.5953716966325214 test f1: 0.6094469304178767
dev acc: 0.8290491104125977 test acc: 0.8277952075004578


100%|██████████| 300/300 [00:07<00:00, 40.06it/s]


dev f1: 0.600757300652328 test f1: 0.6061732920367184
dev acc: 0.8309299945831299 test acc: 0.8270637392997742


100%|██████████| 150/150 [00:03<00:00, 39.90it/s]


In [ ]:
evaluate(dev_loader, model, metric)